# 온라인 어그멘테이션 함수 

In [ ]:


#온라인 어그멘테이션
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam
    
def mixup_loss(loss_fn, pred, labels_a, labels_b, lam):
    return lam * loss_fn(pred, labels_a) + (1 - lam) * loss_fn(pred, labels_b)


def cutout(x, n_holes=1, length=50):
    """
    텐서에 cutout을 적용합니다.
    Args:
        x: 입력 텐서 (B, C, H, W)
        n_holes: 구멍의 개수
        length: 구멍의 길이
    """
    x = x.clone()
    b, c, h, w = x.shape
    
    for i in range(b):
        # 각 이미지마다 mask 생성
        mask = torch.ones((h, w), device=x.device)
        
        for _ in range(n_holes):
            # 랜덤 위치 선택
            y = torch.randint(h, (1,), device=x.device)
            x_pos = torch.randint(w, (1,), device=x.device)
            
            # 영역 계산
            y1 = torch.clamp(y - length // 2, 0, h)
            y2 = torch.clamp(y + length // 2, 0, h)
            x1 = torch.clamp(x_pos - length // 2, 0, w)
            x2 = torch.clamp(x_pos + length // 2, 0, w)
            
            # 마스크에 구멍 뚫기
            mask[y1:y2, x1:x2] = 0
        
        # 모든 채널에 마스크 적용
        mask = mask.expand(c, h, w)
        x[i] = x[i] * mask
    
    return x

def cutmix(x, y, beta=1.0):
    """배치 단위로 cutmix를 적용합니다."""
    batch_size = x.size()[0]
    lam = np.random.beta(beta, beta)
    
    # 랜덤하게 이미지 인덱스를 섞음
    rand_index = torch.randperm(batch_size, device=x.device)
    
    # target a와 b
    y_a = y
    y_b = y[rand_index]
    
    # 이미지 크기
    _, _, h, w = x.size()
    
    # random 영역 선택
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(w * cut_rat)
    cut_h = int(h * cut_rat)
    
    # 랜덤 중심점
    cx = torch.randint(w, (1,), device=x.device)
    cy = torch.randint(h, (1,), device=x.device)
    
    # 영역 좌표
    x1 = torch.clamp(cx - cut_w // 2, 0, w)
    x2 = torch.clamp(cx + cut_w // 2, 0, w)
    y1 = torch.clamp(cy - cut_h // 2, 0, h)
    y2 = torch.clamp(cy + cut_h // 2, 0, h)
    
    # 이미지 혼합
    mixed_x = x.clone()
    mixed_x[:, :, y1:y2, x1:x2] = x[rand_index, :, y1:y2, x1:x2]
    
    # 면적 비율 계산
    lam = 1 - ((x2 - x1) * (y2 - y1) / (w * h))
    
    return mixed_x, y_a, y_b, lam


# 온라인 어그멘테이션 train에 적용시킬 때 

In [ ]:
# 에폭당 학습을 위한 함수
criterion = nn.CrossEntropyLoss() 

def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train() # 모델을 학습 모드로 설정
    train_loss = 0
    preds_list = [] # 예측 결과 리스트 초기화
    targets_list = [] # 타겟 리스트 초기화

    pbar = tqdm(loader) # 진행 상황을 표시하기 위한 tqdm    
    for batch_idx, (image, targets) in enumerate(pbar): 
        image = image.to(device) # 이미지 텐서를 지정한 장치로 이동
        targets = targets.to(device) # 타겟 텐서를 지정한 장치로 이동

        model.zero_grad(set_to_none=True) # 그래디언트 초기화

        # augmentation 선택: 15배치마다 순환 (5: mixup, 10: cutout, 15: cutmix)
        if (batch_idx + 1) % 9 == 3:  # mixup
            mixed_images, targets_a, targets_b, lam = mixup_data(image, targets)
            preds = model(mixed_images)
            loss = mixup_loss(criterion, preds, targets_a, targets_b, lam)
                
            # 평가를 위한 원본 이미지 예측
            with torch.no_grad():
                real_preds = model(image)
                preds_list.extend(real_preds.argmax(dim=1).detach().cpu().numpy())
                targets_list.extend(targets.detach().cpu().numpy())

        elif (batch_idx + 1) % 9 == 6:  # cutout
            cutout_images = cutout(image, n_holes=1, length=30)
            preds = model(cutout_images)
            loss = criterion(preds, targets)
                
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

        elif (batch_idx + 1) % 9 == 0:  # cutmix
            mixed_images, targets_a, targets_b, lam = cutmix(image, targets)
            preds = model(mixed_images)
            loss = mixup_loss(criterion, preds, targets_a, targets_b, lam)
                
            # 평가를 위한 원본 이미지 예측
            with torch.no_grad():
                real_preds = model(image)
                preds_list.extend(real_preds.argmax(dim=1).detach().cpu().numpy())
                targets_list.extend(targets.detach().cpu().numpy())

        # 일반적인 학습
        else:
            preds = model(image) # 모델 예측
            loss = loss_fn(preds, targets) # 손실 계산
                   
        loss.backward() # 역전파
        optimizer.step() # 최적화 단계

        train_loss += loss.item() # 손실 누적
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy()) # 예측 결과 추가
        targets_list.extend(targets.detach().cpu().numpy()) # 타겟 추가

        pbar.set_description(f"Loss: {loss.item():.4f}") # 진행 바에 손실 출력

    train_loss /= len(loader) # 평균 손실 계산
    train_acc = accuracy_score(targets_list, preds_list) # 정확도 계산
    train_f1 = f1_score(targets_list, preds_list, average='macro') # F1 점수 계산

    ret = {
        "train_loss": train_loss, # 평균 손실
        "train_acc": train_acc, # 정확도
        "train_f1": train_f1, # F1 점수
    }

    return ret # 결과 반환